In [256]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
import transformers
# import impact_ner_tokens as get_tokens
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np

import torch
print(torch.cuda.is_available())

label_list = ['O','B-IMPACT','I-IMPACT','B-AFFECTED','I-AFFECTED','B-SEVERITY','I-SEVERITY','B-LOCATION','I-LOCATION','B-MODIFIER','I-MODIFIER']

tokenizer = AutoTokenizer.from_pretrained('./models/roberta-base-new-bio-updated-ner-6.model/')

False


In [257]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./models/roberta-base-new-bio-updated-ner-6.model/', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [258]:

tokens = tokenizer('''#BalochistanEarthquaeke At least 25 people have reportedly been killed as a second powerful earthquake has struck occupied #Balochistan''')
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

torch.Size([1, 30])

In [259]:
model = AutoModelForTokenClassification.from_pretrained('./models/roberta-base-new-bio-updated-ner-6.model/', num_labels=len(label_list))

In [260]:
import torch
preds = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))

In [261]:
preds = torch.argmax(preds.logits.squeeze(), axis=1)

In [262]:
sentence = '''#BalochistanEarthquaeke At least 25 people have reportedly been killed as a second powerful earthquake has struck occupied #Balochistan'''

In [263]:
len(sentence.split(' '))

18

In [264]:
preds.size()

torch.Size([30])

In [265]:
len(tokens['input_ids'])

30

In [266]:
words = tokenizer.batch_decode(tokens['input_ids'])

In [267]:
value_preds = [label_list[i] for i in preds]

In [268]:
import pandas as pd
pd.DataFrame({'ner': value_preds, 'words': words}).to_csv('./model_output/roberta-base-new-bio-updated-ner-6.csv')